# Vektorisierung

In [14]:
import pandas as pd

Dataset "train_cleaned" importieren

In [15]:
df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)

In [16]:
df.head()

,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
id,,,,,,
1,0,@user when a father is dysfunctional and is s...,father selfish drag kid run,1,['#run'],NaN
2,0,@user @user thanks for #lyft credit i can't us...,thank lyft credit use cause offer van,2,"['#lyft', '#disapointed', '#getthanked']",NaN
3,0,bihday your majesty,bihday majesty,0,[],NaN
4,0,#model i love u take with u all the time in ...,model take time mobile phone kiss sunglass mou...,0,['#model'],":mobile_phone:,:kissing_face_with_smiling_eyes..."
5,0,factsguide: society now #motivation,factsguide society motivation,0,['#motivation'],NaN


Da es leere tweet_cleaned gibt müssen diese hier entfernt werden. Ansonsten werfen alle 3 Methoden Fehler

In [17]:
df = df[df['tweet_cleaned'].notna()]

## Bag of Words

In [5]:
# Doku: https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() # CountVectorizer(binary=True) würde nur 0 und 1 ausgeben
X_bow = vectorizer.fit_transform(df.tweet_cleaned)

In [6]:
print(vectorizer.get_feature_names_out())

['aamp' 'aap' 'ab' ... 'zootopia' 'zoro' 'zzz']


In [7]:
print(X_bow.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
# Erster Test NB

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

y = df.label
clf = MultinomialNB()

clf.fit(X_bow,y)

ypred = clf.predict(X_bow)

print("Accuracy: ", accuracy_score(y, ypred))

Accuracy:  0.9523671742196824


#### Ergebnis Bag of Word in X_bow

## TF-IDF

In [11]:
# Doku https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df.tweet_cleaned)

In [12]:
print(tfidf_vectorizer.get_feature_names_out())

['aamp' 'aap' 'ab' ... 'zootopia' 'zoro' 'zzz']


In [13]:
print(X_tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
# Erster Test NB

y = df.label
clf = MultinomialNB()

clf.fit(X_tfidf,y)

ypred_tfidf = clf.predict(X_tfidf)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

Accuracy:  0.954609569966364


#### Ergebnis TF-IDF in X_tfidf

## Word2Vec

In [15]:
from nltk import word_tokenize
from gensim.models import Word2Vec
import numpy as np

data = df['tweet_cleaned'].map(word_tokenize)

In [16]:
data

id
1                        [father, selfish, drag, kid, run]
2            [thank, lyft, credit, use, cause, offer, van]
3                                        [bihday, majesty]
4        [model, take, time, mobile, phone, kiss, sungl...
5                        [factsguide, society, motivation]
                               ...                        
31956    [less, week, grin, sweat, fold, hand, medium, ...
31957         [fishing, tomorrow, wait, first, time, year]
31958                                    [eat, youuu, red]
31959    [see, turner, try, wrap, genuine, hero, like, ...
31960    [listen, sad, song, monday, morning, otw, work...
Name: tweet_cleaned, Length: 27649, dtype: object

In [17]:
w2v = Word2Vec(data,window=4, min_count=1, sg=0) 
# sg=1: Skip-Gram
# sg=0: Continuous Bag of Words

In [18]:
w2v.train(data, total_examples=len(df['tweet_cleaned']), epochs=20)
# epochs = Anzahl Iterationen

(3895292, 4126720)

In [19]:
w2v.wv.most_similar(positive="motivation")

[('inspiration', 0.8603976368904114),
 ('positivity', 0.8576184511184692),
 ('inspirational', 0.778499186038971),
 ('quoteoftheday', 0.7686036825180054),
 ('motivate', 0.7532286047935486),
 ('belief', 0.7525449395179749),
 ('entrepreneur', 0.7442077994346619),
 ('selflove', 0.7372958660125732),
 ('personaldevelopment', 0.7356842160224915),
 ('clarity', 0.7265296578407288)]

In [20]:
w2v.wv.most_similar(negative="motivation")

[('meet', 0.39135491847991943),
 ('august', 0.3650570809841156),
 ('th', 0.35828283429145813),
 ('poland', 0.34213826060295105),
 ('july', 0.34154877066612244),
 ('yr', 0.3362632095813751),
 ('st', 0.3296425938606262),
 ('rd', 0.32583528757095337),
 ('half', 0.3231406509876251),
 ('date', 0.3174822926521301)]

In [21]:
size = 100

def w2v_vector(tokenized_tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokenized_tweet:
        try:
            vec += w2v.wv[word].reshape((1, size))
            count += 1
        except KeyError:
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [22]:
X_w2v = np.zeros((len(data),size))

In [24]:
for i in range(len(data)):
    X_w2v[i,:] = w2v_vector(data.iloc[i],size)

In [25]:
X_w2v

array([[-0.32555764,  0.39498052, -0.32812717, ..., -0.99470037,
        -0.20116415, -0.89298161],
       [-0.3282834 ,  0.54676195,  0.00112542, ..., -0.26532286,
        -0.15750324, -0.49165652],
       [ 0.31196176,  0.22025196, -0.50505333, ..., -0.36300076,
         0.16350489,  0.62725972],
       ...,
       [-0.63985382, -0.65229094,  0.92911165, ..., -0.65501505,
         0.32828084, -0.16535795],
       [-0.30423937,  0.46370349,  0.08884029, ..., -0.0956155 ,
         0.01651742,  0.07102602],
       [-0.63813522,  0.58901546, -0.77321871, ...,  0.11527004,
         0.19589269,  0.20339311]])

In [26]:
# Erster Test NB
from sklearn.naive_bayes import GaussianNB

y = df.label
clf = GaussianNB()

clf.fit(X_w2v,y)

ypred_tfidf = clf.predict(X_w2v)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

Accuracy:  0.7570255705450468


#### Ergebnis Word2Vec in X_w2v

## FastText

In [30]:
from gensim.models import FastText 

ft = FastText(data, window = 4)
ft.train(data, total_examples=len(df['tweet_cleaned']), epochs=20)

(3789064, 4126720)

In [31]:
ft.wv.most_similar(positive="motivation")

[('tuesdaymotivation', 0.9570346474647522),
 ('mondaymotivation', 0.9523440599441528),
 ('motivational', 0.9495548009872437),
 ('inspiration', 0.8649970889091492),
 ('affirmation', 0.8257686495780945),
 ('transformation', 0.8194717764854431),
 ('meditation', 0.8178461790084839),
 ('confirmation', 0.7999058961868286),
 ('innovation', 0.7997733950614929),
 ('position', 0.7960103154182434)]

In [32]:
ft.wv.most_similar(negative="motivation")

[('yr', 0.3835121691226959),
 ('v', 0.3463435471057892),
 ('hi', 0.34137162566185),
 ('gator', 0.3400598168373108),
 ('sit', 0.32863670587539673),
 ('cox', 0.3283883035182953),
 ('xbox', 0.3270142078399658),
 ('die', 0.32616955041885376),
 ('snatch', 0.3259742558002472),
 ('bro', 0.32236719131469727)]

In [33]:
size = 100

def ft_vector(tokenized_tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokenized_tweet:
        try:
            vec += ft.wv[word].reshape((1, size))
            count += 1
        except KeyError: 
                         
            continue
    if count != 0:
        vec /= count
    return vec

In [34]:
X_ft = np.zeros((len(data),size))

In [35]:
for i in range(len(data)):
    X_ft[i,:] = ft_vector(data.iloc[i],size)

In [36]:
X_ft

array([[-0.76547772, -0.21383475, -0.40605466, ..., -0.95944892,
        -0.78261749,  0.35719954],
       [-0.5774988 , -0.04261827, -0.24013078, ..., -0.04634736,
         0.61478294,  0.17527176],
       [ 0.81800085,  0.57611634,  0.08267095, ..., -1.20095155,
        -0.38159819, -0.06786457],
       ...,
       [-0.16428977, -0.29616243,  0.1838846 , ..., -0.87193196,
         0.509852  , -0.64583018],
       [-0.16578822, -0.05996855, -0.39674566, ...,  0.05825424,
         0.52146263,  0.31040981],
       [-0.51245216,  0.18201927, -0.49733237, ..., -0.3617763 ,
        -0.34870465, -0.19760916]])

In [37]:
# Erster Test NB
from sklearn.naive_bayes import GaussianNB

y = df.label
clf = GaussianNB()

clf.fit(X_ft,y)

ypred_tfidf = clf.predict(X_ft)

print("Accuracy: ", accuracy_score(y, ypred_tfidf))

Accuracy:  0.7548555101450324


#### Ergebnis FastText in X_ft

In [38]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from gensim.models import FastText
from sklearn.model_selection import train_test_split

In [13]:
%run ../sven/vf.py

In [36]:
X_base = df.tweet_cleaned
y_base =df.label

In [26]:
from sklearn.model_selection import train_test_split

In [32]:
# Doku: https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.train_test_split.html
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3, random_state=42)

In [72]:
def split(df):
    
    X_base = df.tweet_cleaned
    y_base =df.label
    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3, random_state=42)
    
    return X_train_base, X_test_base, y_train_base, y_test_base
    

In [76]:
X_train_base_e.shape

NameError: name 'X_train_base_e' is not defined

In [34]:
y_train_base.shape

(19354,)

In [35]:
X_test_base.shape

(8295,)

In [37]:
y_test_base.shape

(8295,)

### tfidf

In [40]:
tfidf_vectorizer = TfidfVectorizer()

In [42]:
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_base)
X_test_tfidf = tfidf_vectorizer.transform(X_test_base)

In [43]:
X_train_tfidf.shape

(19354, 6293)

In [44]:
X_test_tfidf.shape

(8295, 6293)

### bow

In [45]:
bow_vectorizer = CountVectorizer()

In [47]:
X_train_bow = bow_vectorizer.fit_transform(X_train_base)

In [48]:
X_test_bow = bow_vectorizer.fit_transform(X_test_base)

In [50]:
X_train_bow.shape

(19354, 6293)

In [51]:
X_test_bow.shape

(8295, 5672)

In [59]:
split

<function __main__.split()>

In [77]:
def vectorize_bow():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]
    
    X_base = df.tweet_cleaned
    y_base =df.label
    
    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3, random_state=42)
    
    bow_vectorizer = CountVectorizer()
    
    X_train_bow, = bow_vectorizer.fit_transform(X_train_base)
    X_test_bow = bow_vectorizer.fit_transform(X_test_base)
    
    return X_train_bow, X_test_bow, y_train_base, y_test_base
    
    
    
    


In [78]:
def vectorize_tfidf():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]
    
    X_base = df.tweet_cleaned
    y_base =df.label
    
    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3, random_state=42)
    
    tfidf_vectorizer = TfidfVectorizer()
    
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_base)
    X_test_tfidf = tfidf_vectorizer.transform(X_test_base)
    
    return X_train_tfidf, X_test_tfidf, y_train_base, y_test_base

    

In [ ]:
def vectorize_w2v():
    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]

    X_base = df.tweet_cleaned
    y_base = df.label

    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3,
                                                                            random_state=42)

    X_train_base_tokenized = X_train_base.map(word_tokenize)
    X_test_base_tokenized = X_test_base.map(word_tokenize)

    w2v = Word2Vec(X_train_base, window=4, min_count=1, sg=0)
    w2v.train(X_train_base_tokenized, total_examples=len(X_train_base), epochs=20)

    def w2v_vector(tokenized_tweet, size):
        vec = np.zeros(size).reshape((1, size))
        count = 0
        for word in tokenized_tweet:
            try:
                vec += w2v.wv[word].reshape((1, size))
                count += 1
            except KeyError:

                continue
        if count != 0:
            vec /= count
        return vec

    size = 100
    X_train_w2v = np.zeros((len(X_train_base_tokenized), size))
    for i in range(len(X_train_base_tokenized)):
        X_train_w2v[i, :] = w2v_vector(X_train_base_tokenized.iloc[i], size)

    X_test_w2v = np.zeros((len(X_test_base_tokenized), size))
    for i in range(len(X_test_base_tokenized)):
        X_test_w2v[i, :] = w2v_vector(X_test_base_tokenized.iloc[i], size)

    return X_train_w2v, X_test_w2v, y_train_base, y_test_base

In [ ]:
def vectorize_ft():
    def ft_vector(tokenized_tweet, size):
        vec = np.zeros(size).reshape((1, size))
        count = 0
        for word in tokenized_tweet:
            try:
                vec += ft.wv[word].reshape((1, size))
                count += 1
            except KeyError:

                continue
        if count != 0:
            vec /= count
        return vec

    df = pd.read_csv('../../../data/twitter_hate-speech/train_cleaned.csv', index_col=0)
    df = df[df['tweet_cleaned'].notna()]

    X_base = df.tweet_cleaned
    y_base = df.label

    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, test_size=0.3,
                                                                            random_state=42)

    X_train_base_tokenized = X_train_base.map(word_tokenize)
    X_test_base_tokenized = X_test_base.map(word_tokenize)

    ft = FastText(X_train_base, window=4)
    ft.train(X_train_base_tokenized, total_examples=len(X_train_base), epochs=20)

    size = 100
    X_train_ft = np.zeros((len(X_train_base_tokenized), size))
    for i in range(len(X_train_base_tokenized)):
        X_train_ft[i, :] = ft_vector(X_train_base_tokenized.iloc[i], size)

    X_test_ft = np.zeros((len(X_test_base_tokenized), size))
    for i in range(len(X_test_base_tokenized)):
        X_test_ft[i, :] = ft_vector(X_test_base_tokenized.iloc[i], size)

    return X_train_ft, X_test_ft, y_train_base, y_test_base
